In [1]:
from tqdm import tqdm
import pandas as pd
import os
import gzip
import zipfile
import shutil
from time import sleep
import pickle
import tarfile
import bz2
import sys
import numpy as np

In [2]:
df_marks = pd.read_csv('train.marks.tsv', delimiter='\t', header=None)
df_marks = df_marks.rename(columns={0: "QueryId", 1: "DocumentId"})
df_marks = df_marks.drop(columns=[2])
df_example = pd.read_csv('sample.csv')
all_groups = df_marks.append(df_example)
all_groups = all_groups.reset_index()
all_groups = all_groups.drop(columns=['index'])
all_groups = all_groups.sort_values(by=['QueryId', 'DocumentId']).reset_index()
all_groups = all_groups.drop(columns=['index'])
rev_frame = all_groups.sort_values(by=['DocumentId', 'QueryId']).reset_index()
rev_frame = rev_frame.drop(columns=['index'])

In [3]:
host_skelet = dict()
q_ids = all_groups['QueryId'].unique()
for qid in q_ids:
    host_skelet[qid] = set()

In [4]:
listOfFiles = os.listdir('rh_host_data_upd')
idx = [int(str2[3:-4]) for str2 in listOfFiles]
idx.sort()
for idx1 in tqdm(idx):
    with zipfile.ZipFile('rh_host_data_upd/rh_{}.zip'.format(idx1)) as myzip:
        with myzip.open('rh_{}.pickle'.format(idx1), 'r') as file:
            obj = pickle.load(file)
            for session in obj:
                if session[0]>-1:
                    host_skelet[session[0]].add(idx1)

100%|██████████| 1900/1900 [51:41<00:00,  1.63s/it]  


In [5]:
with open('host_skelet.pickle', 'wb') as file:
    pickle.dump(host_skelet, file)

In [6]:
1+2

3

In [8]:
for key in tqdm(host_skelet):
    if len(host_skelet[key])>0:
        data = []
        for num in host_skelet[key]:
            with zipfile.ZipFile('rh_host_data_upd/rh_{}.zip'.format(num)) as myzip:
                with myzip.open('rh_{}.pickle'.format(num), 'r') as file:
                    obj = pickle.load(file)
                    tmp = list(filter(lambda x: x[0]==key, obj))
                    data.extend(tmp)
        data = tuple(data)
        with open('host_query/qh_{}.pickle'.format(key), 'wb') as file:
            pickle.dump(data, file)

100%|██████████| 6311/6311 [4:53:23<00:00,  2.79s/it]   


In [9]:
1+2

3